# Scraping Wikipedia Celebrity Deaths

Creating a notebook to produce the dataset found at the [Kaggle Celebrity Deaths Page](https://www.kaggle.com/hugodarwood/celebrity-deaths).

Attempting to replace the current dataset since it isn't complete (up-to-date) since there's no notebook to run to get up-to-date information and it has bad parses for some of the fields.

This notebook isn't perfect either but does a better job.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import re
import requests
import json

from os import listdir
from os.path import isfile, join

from bs4 import BeautifulSoup as bs

In [ ]:
month_to_num = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

Iterators:

In [ ]:
year_list = range(2004,2017)
type(year_list)

### [DON'T RUN AGAIN] Scrape all monthly death pages

Scrape all death pages and store into the '../out/raw_pages' directory

In [ ]:
"""
for year in year_list:
    for month in month_to_num.keys():
        filename = str(year) + '_' + str(month_to_num[month]) + '_deaths.json'
        url = ('https://en.wikipedia.org/w/api.php?action=query&titles=Deaths_in_' +  
               month + '_' + str(year) + 
               '&prop=revisions&rvprop=content&format=json')
        content = requests.get(url).json()
        with open("../out/raw_pages/" + filename, "wb") as outfile:
            json.dump(content, outfile)         
"""
''

### Helper functions

In [ ]:
desc_death_re = re.compile('(.*?),? ?((?:.+)*?). (?:.*?)?', re.DOTALL|re.MULTILINE)
death_clean_no_url_re = re.compile('\s?(.[^<]+)\.? ?(?=<|\[?http)(?:.*)?$')
# death_clean_no_url_re = re.compile('\s?\w(.[^<])+[.]?(?:<|\[?http)(?:.*)?$')

"""
Input: single text string to be processed

Output: list of strings of length two
  - first string is description of person
  - second string is cause of death 
  (last clause of input when more than one comma in field)
  
"""
def get_description_and_death(text):
    text_no_url = text
    if ('http' in text_no_url) or ('<ref' in text_no_url):
        try:
            text_no_url = death_clean_no_url_re.match(text).groups()[0]
        except AttributeError:
            print text
            raw_input("Press enter to continue")
            raw_input("Press enter to continue")
    text_parts = text_no_url.replace('=','').split(',')
    num_parts = len(text_parts)
    if num_parts == 0:
        return ['', '']
    elif num_parts == 1:
        return text_parts + ['']
    elif num_parts == 2:
        return [",".join(text_parts)] + [text_parts[1]]
    else:
        return ([",".join(text_parts[:-1])] + [text_parts[-1]])
                #[death_clean_no_url_re.match(text_parts[-1]).groups()[0]])

In [ ]:
"""
Runs get_description_and_death on the last element of a list
"""
def add_description_and_death(entry_list):
    return entry_list[:-1] + get_description_and_death(entry_list[-1])

Testing functions:

In [ ]:
get_description_and_death('English singer-songwriter ("[[Won\'t Somebody Dance with Me]]"), brain haemorrhage.<ref>[http://www.bbc.co.uk/news/entertainment-arts-29457228 Singer Lynsey De Paul dies aged 64]</ref>')

In [ ]:
"""
This is for input as a list cleaning a particular list entry. 
See below for the pandas dataframe version

Inputs: list, index number, character pattern to be replaced

Outputs: list with entry in index number cleaned of the character pattern to be replaced
"""
def clean_text(text_list, idx_num, chars):
    text_list_out = text_list
    text_list_out[idx_num] = text_list[idx_num].replace(chars, '')
    return text_list_out

In [ ]:
mo_yr_key_re = re.compile('(\d+)_(\d+).*?')
name_age_re = re.compile('\[\[(.*?)\]\], (\d+), (.+)?$', re.MULTILINE)

"""
Inputs: month-year key string, text entry string

Outputs: list of month, year, name, and age
"""
def parse_month_year_name_age(my_key, text_entry):
    return (list(re.match(mo_yr_key_re, my_key).groups()) +
            list(re.match(name_age_re, text_entry.replace('\n', '')).groups()))

In [ ]:
"""
Find links and convert them to the displayed text
"""
link_re = re.compile('\[\[([^\|(?:\]\])]*)(?=\||\]\])', re.DOTALL)

# need to check if contains link
def extract_link_text(link_block):
    link_present = link_re.search(link_block)
    if link_present:
        return link_present.groups()
    return link_block

In [ ]:
extract_link_text('asdf asdf [[1asdf|2]]')

In [ ]:
link_all_re = re.compile('(\[\[([^(\]\])]*)(\]\]))')

"""
Used to be messy, not anymore!
"""
def link_only(matchobj):
    cleaned_text = extract_link_text(matchobj.groups()[0])[0]
    return cleaned_text

def remove_link_text(text_block):
    return re.sub(link_all_re, link_only, text_block)

In [ ]:
remove_link_text('asdf asdf [[ab]] [[cd|efg]]')

In [51]:
def remove_end_period(text):
    return re.sub('\. ?$','',text)

def remove_beginning_space(text):
    return re.sub('^ +','',text)

In [52]:
remove_beginning_space('   asdf')

'asdf'

In [53]:
"""
for a single entry in the dataframe
"""
def text_clean(text):
    return remove_beginning_space(
        remove_end_period(
            remove_link_text(text)
        ).replace('[','').replace(']','')
    )

### Re-read and process scraped file data locally

In [54]:
jsonfiles = [f for f in listdir('../out/raw_pages/') if (isfile(join('../out/raw_pages/', f)) and f[-4:] == 'json')]

In [121]:
month_year_pages = {}

for jfilename in jsonfiles:
    my_key = "_".join(re.match(mo_yr_key_re, jfilename).groups())
    with open('../out/raw_pages/' + jfilename, 'rb') as infile:
        contents = infile.read()
        infile.close()
    month_year_pages[my_key] = json.loads(contents)['query']['pages'].values()[0]['revisions'][0]['*']

In [122]:
counter = 0
for value in month_year_pages.values():
    new_entries = len(value.encode('utf-8').rstrip().split('*'))
    counter += new_entries
counter

57927

Should combine cell below with cell above (later).

In [123]:
for my_key in month_year_pages.keys():
    month_year_pages[my_key] = [
        add_description_and_death(
            parse_month_year_name_age(my_key, entry)
        )
        for entry in month_year_pages[my_key].encode('utf-8').rstrip().split('*')
        if re.match(name_age_re, entry.replace('\n', ''))]

Make dataframe:

In [124]:
df_full = pd.DataFrame(columns=['year','month','name','age','desc','cause_of_death'])
print df_full.shape
df_full.head()

(0, 6)


,year,month,name,age,desc,cause_of_death


In [125]:
for entry in month_year_pages.values():
    df_sub = pd.DataFrame(entry, columns=['year','month','name','age','desc','cause_of_death'])
    df_full = pd.concat([df_full, df_sub], axis=0)

In [129]:
print df_full.shape
df_full.head()

(49051, 6)


,year,month,name,age,desc,cause_of_death
0,2014,10,Lynsey de Paul,64,"English singer-songwriter (""Won't Somebody Dan...",brain haemorrhage
1,2014,10,Maurice Hodgson|Sir Maurice Hodgson,94,British business executive,
2,2014,10,Shlomo Lahat,86,"Israeli general and politician, Mayor of Tel A...",lung infection
3,2014,10,José Martínez (infielder)|José Martínez,72,"Cuban baseball player (Pittsburgh Pirates), co...",Chicago Cubs) and executive (Atlanta Braves)
4,2014,10,Oluremi Oyo,61,"Nigerian journalist, cancer",cancer


In [130]:
df_full.iloc[4].values[5]

'cancer'

### Further Processing

#### Reminder: Strip links, quotes, and brackets before extracting Nationality text

In [128]:
df_full['desc'] = df_full.desc.map(text_clean)
df_full['cause_of_death'] = df_full.cause_of_death.map(text_clean)

#### Extract Nationality

Extracting nationality text as well as possible by taking the first consecutive capitalized words in the description. 'Olympic' and similar capitalized words might throw this off.

In [131]:
natl_pattern1 = re.compile(' ?((?:[A-Z][^\s]+ ?)+) ', re.UNICODE)

def get_nationality_text(desc_text):
    try:
        # get rid of url links
        return natl_pattern1.match(desc_text.strip('[')).groups()[0]
    except AttributeError, e:
        print desc_text

In [132]:
natl_pattern1.match('Native American asdf asdfasdf ').groups()

('Native American',)

In [133]:
df_full['nationality'] = df_full.desc.map(get_nationality_text)

ni-Vanuatu international footballer
chief of web security at AOL, pancreatic cancer 
<!-- date of birth unknown -->British artist. 
former Canadian Football League player and NHL referee. 
former Soviet FIFA World Cup footballer & title-winning coach for Dynamo Kiev 
former German Nazi SS officer. 
former Second Deputy Prime Minister of Singapore, heart failure. 
former professional wrestling promoter, natural causes 
last American survivor of the ''RMS Titanic'' sinking, died in sleep. 
former CEO of Firestone Tire and Rubber Company, heart attack. 
fled the U.S. with granddaughter in Elizabeth Morgan case custody battle 
lawyer & professor at New York Law School 
founder of Ninety Nine Restaurant & Pub, cancer. 
first Asian-American to be editor of a major American daily newspaper, the ''St. Louis Post-Dispatch''
first Australian Defence Force serviceperson killed in Iraq. 
humanitarian worker and charity activist
scholar and journalist, Director of the Department of Culture (since 2

In [134]:
def get_wiki_url(name_text):
    return name_text.split('|')[0].strip('[').strip(']')

In [135]:
df_full['name'] = df_full.name.map(get_wiki_url)

In [136]:
df_full.head()

,year,month,name,age,desc,cause_of_death,nationality
0,2014,10,Lynsey de Paul,64,"English singer-songwriter (""Won't Somebody Dan...",brain haemorrhage,English
1,2014,10,Maurice Hodgson,94,British business executive,,British
2,2014,10,Shlomo Lahat,86,"Israeli general and politician, Mayor of Tel A...",lung infection,Israeli
3,2014,10,José Martínez (infielder),72,"Cuban baseball player (Pittsburgh Pirates), co...",Chicago Cubs) and executive (Atlanta Braves),Cuban
4,2014,10,Oluremi Oyo,61,"Nigerian journalist, cancer",cancer,Nigerian


### Creating a new table for birth and death dates

Converting to dict to get page_size, date of birth, and date of death

In [142]:
len(df_full.name.values)

49051

In [143]:
names_only = dict(zip(df_full.name.values, [['', '', ''] for name in df_full.name.values]))

In [ ]:
%%time
counter = 0
for name in names_only.keys():
    if names_only[name][0] == '':
        counter += 1
        #names_only[name][0] = get_page_size(name)
print counter

In [ ]:
%%time
for name in names_only.keys():
    names_only[name][1] = get_life_death(name)

## Further attributes outside of the death summary Wiki page

### Article length: Fame Score

Get Wikipedia URL for getting fame score i.e. size of article:

Get page metadata:

In [146]:
base_url_prefix = 'https://en.wikipedia.org/w/api.php?action=query&format=json&titles=' 
base_url_suffix = '&prop=revisions&rvprop=size'

In [ ]:
def get_page_size(wiki_name):
    try:
        size_url = base_url_prefix + wiki_name.replace(' ','_') + base_url_suffix
        size_page = requests.get(size_url)
        return size_page.json()['query']['pages'].values()[0]['revisions'][0]['size']
    except KeyError:
        print wiki_name

### Birth and death data

Getting birthday data. It's probably helpful to see [Stack Overflow thread](https://stackoverflow.com/questions/12250580/parse-birth-and-death-dates-from-wikipedia)

In [ ]:
base_bday_prefix = 'https://en.wikipedia.org/w/api.php?action=query&format=json&titles='
base_bday_suffix = '&prop=revisions&rvprop=content&rvsection=0'

def check_if_number(elem):
    try:
        return type(int(elem)) == int
    except ValueError:
        return False

    
def get_only_ints(date_list):
    return [e for e in date_list if check_if_number(e) > 0]

life_death_counter = [0]
no_birth_death_urls = []

def get_life_death(name, extract_bday=True):
    life_death_counter[0] += 1
    try:        
        entry_url = base_bday_prefix + name.replace(' ','_') + base_bday_suffix
        entry_text = requests.get(entry_url).json()['query']['pages']
        entry_text = str(entry_text.values()[0]['revisions'][0]['*'].encode('utf-8'))
        
        if extract_bday:
            pattern = re.compile('.*?\{{2}(?:B|b)irth (?:D|d)ate(.+?)\}{2}', 
                                 re.MULTILINE|re.DOTALL)
        else:
            pattern = re.compile('.*?\{{2}(?:D|d)eath (?:D|d)ate(.+?)\}{2}', 
                                 re.MULTILINE|re.DOTALL)

        date_re = re.match(pattern, entry_text)
        date_data = "-".join(get_only_ints(date_re.groups()[0].strip(" ").split('|'))[:3])
        
        #print date_data
        if life_death_counter[0] % 100 == 1:
            print date_data
        return date_data
    
    except (AttributeError, KeyError, ValueError) as e:
        no_birth_death_urls.append(entry_url)

Birthday

In [38]:
%%time
#df_full['date_of_birth'] = df_full.name.map(get_life_death)
get_life_death(df_full.iloc[0][2])

1948-6-11
CPU times: user 44.2 ms, sys: 13.5 ms, total: 57.7 ms
Wall time: 401 ms


'1948-6-11'

In [45]:
df_full.iloc[0][5]

' brain haemorrhage'

Deathday

In [ ]:
%%time
df_full['date_of_death'] = df_full.name.map(lambda name: get_life_death(name, extract_bday=False))

## Write out file

In [37]:
df_full.to_csv('../out/celeb_deaths_wikipedia_full_1.csv', index=False)

It would be useful to plot missing data by page_size i.e. fame or "importance."